# LMS with Kernel Trick

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
file = pd.read_csv("california_housing_train.csv")
file

As you can see, there are 12 different input features, but we will remove some, and use only a few others. Also, my pc can't handle this load :P so I'm reducing the dataset size. I'll use only $10k$ data sets. Also, we'll split that into training set and testing set!

In [ ]:
#selecting only the required columns
# useful_index = [3, 4, 5, 6, 7, 8, 10]
# file = file.iloc[:, useful_index]

# getting training set and test set
file = file.sample(frac=1, random_state=1)

# useable data size
usable_dataset_size = 2000

# get rid of unnecessary data
file = file[:usable_dataset_size]

## Objective: To predict apparent temperature from other features

### 1. Arrange the data in appropriate format

In [ ]:
# Select the columns
X = file.iloc[:, :-1]
output_index = [-1]
Y = file.iloc[:, output_index]

# Convert to numpy into suitable format
X = X.to_numpy()
n = X.shape[0]
d = X.shape[1]

Y = Y.to_numpy()

### 3. Construct the training and prediction function

The following function calculates the kernel $\langle \phi\left(x\right), \phi\left(y\right) \rangle$

In [ ]:
def make_kernel(X, degree=1):
    """K is the matrix of dot product. This applies kernel function to the matrix"""
    K = np.matmul(X, X.T)
    result = np.zeros(K.shape)
    for i in range(0, degree+1):
        result += np.power(K, i)
    return result

The following function trains the data

In [ ]:
def lms_with_kt(X, Y, alpha, degree = 1, num_iters = 1000):
    """X: nxd vector, Y: nx1 vector, beta: nx1 zero vector, alpha: number, degree: number"""
    # normalize x
    x_min = X.min(axis = 0, keepdims=True)
    x_max = X.max(axis = 0, keepdims=True)
    X = (X - x_min) / (x_max - x_min)
    n = X.shape[0]
    
    # add the column of 1 in the front
    # X = np.hstack((np.ones((n, 1)), X))
    
    # make K_matrix (kernel matrix)
    K = make_kernel(X, degree)
    
    # initialize beta
    beta = np.zeros((n, 1))

    # update beta
    for i in range(num_iters):
        beta += alpha * (Y - np.matmul(K, beta))
    print(beta)
        
    def predict(x):
        """x: 1xd matrix"""
        x_norm = (x - x_min) / (x_max - x_min)
        n_predict = x_norm.shape[0]
        # x_norm = np.hstack((np.ones((n_predict, 1)), x_norm))
        K_for_prediction = np.matmul(X, x_norm.T)
        K_for_prediction = kernel_matrix(K_for_prediction, degree)
        return np.dot(beta.T, K_for_prediction)

    return predict

### 4. Train the data and predict the results

#### 4.1 Format the test set

In [ ]:
# # separate inputs and outputs
# test_set_inputs = test_set.iloc[:, input_index]
# test_set_outputs = test_set.iloc[:, output_index]

# # Convert to numpy
# test_set_inputs = test_set_inputs.to_numpy()
# test_set_outputs = test_set_outputs.to_numpy()

#### 4.2 Training

In [ ]:
predictor = lms_with_kt(X, Y, 0.01, 3, 1000)

#### 4.3 Prediction

In [ ]:
# results = predictor(test_set_inputs)